In [1]:
# from keras.applications.densenet import DenseNet201, DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense
from keras import layers, optimizers, models
import keras
from keras import backend as K
# from keras.applications.densenet import preprocess_input
import time

import matplotlib.pyplot as plt

from keras.models import load_model
from keras.applications import imagenet_utils
from keras.applications.imagenet_utils import _preprocess_symbolic_input
from keras.preprocessing import image

/home/1nnuser/lsdaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
import glob
import pickle
import numpy as np

FEATURES_LOCATION = '/furniture-data/features/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

In [2]:
from keras.applications import VGG16
#Load the VGG model
image_size = 224
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [3]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers:
    layer.trainable = False
    
# for layer in vgg_conv.layers[:-4]:
#     layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f84bd25ae10> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd25af60> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd2060f0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f84bd206630> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd225358> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd1bbdd8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f84bd1711d0> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd1e2358> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd185f28> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd13a588> False
<keras.layers.pooling.MaxPooling2D object at 0x7f84bd1aa6d8> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd14f438> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd0f5278> False
<keras.layers.convolutional.Conv2D object at 0x7f84bd106b38> False
<keras.layers.pooling.MaxPooling2D object at 0x7f84bd129e80> Fa

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               131200    
Total params: 40,537,024
Trainable params: 25,822,336
Non-trainable params: 14,714,688
_________________________________________________________________


In [17]:
X, y = load_data('train')
X_val, y_val = load_data('valid')

from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced',
                                             np.unique(y),
                                             y)

In [5]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10

train_dir = '/furniture-data/train'
val_dir = '/furniture-data/valid'
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 192171 images belonging to 128 classes.
Found 6309 images belonging to 128 classes.


In [12]:
# Fine-tuning optimizer
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.RMSprop(lr=1e-4),
#               metrics=['acc'])

# Pretraining optimizer
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size/10,
      epochs=2*10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size/2,
      verbose=1,
      callbacks=[keras.callbacks.ModelCheckpoint('learnopencv_vgg.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto', period=1)])
 
# Save the model
model.save('learnopencv_vgg_final.h5')

Epoch 1/20
193/192 [==============================] - 1032s 5s/step - loss: 4.2427 - acc: 0.0910 - val_loss: 3.2307 - val_acc: 0.2288

Epoch 00001: val_loss improved from inf to 3.23066, saving model to learnopencv_vgg.h5
Epoch 2/20
193/192 [==============================] - 960s 5s/step - loss: 3.0155 - acc: 0.2793 - val_loss: 2.6937 - val_acc: 0.3530

Epoch 00002: val_loss improved from 3.23066 to 2.69374, saving model to learnopencv_vgg.h5
Epoch 3/20
193/192 [==============================] - 960s 5s/step - loss: 2.5111 - acc: 0.3776 - val_loss: 2.1297 - val_acc: 0.4611

Epoch 00003: val_loss improved from 2.69374 to 2.12975, saving model to learnopencv_vgg.h5
Epoch 4/20
193/192 [==============================] - 955s 5s/step - loss: 2.2013 - acc: 0.4417 - val_loss: 2.0319 - val_acc: 0.5106

Epoch 00004: val_loss improved from 2.12975 to 2.03189, saving model to learnopencv_vgg.h5
Epoch 5/20
193/192 [==============================] - 966s 5s/step - loss: 2.0162 - acc: 0.4833 - val_l

In [18]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size/10,
      epochs=2*10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size/2,
      verbose=1,
      callbacks=[keras.callbacks.ModelCheckpoint('learnopencv_vgg2.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto', period=1)])
 
# Save the model
model.save('learnopencv_vgg_final2.h5')

Epoch 1/20


KeyboardInterrupt: 

In [8]:
loaded_model = keras.models.load_model('learnopencv_vgg2.h5')

In [11]:
for layer in loaded_model.layers[0].layers:
    layer.trainable = False
    print(layer, layer.trainable)

for layer in loaded_model.layers[0].layers[-8:]:
    layer.trainable = True
    print(layer, layer.trainable)
    
print('')
    
for layer in loaded_model.layers[0].layers[:]:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f849875e4a8> False
<keras.layers.convolutional.Conv2D object at 0x7f849875e518> False
<keras.layers.convolutional.Conv2D object at 0x7f849875e780> False
<keras.layers.pooling.MaxPooling2D object at 0x7f849875e588> False
<keras.layers.convolutional.Conv2D object at 0x7f849875eac8> False
<keras.layers.convolutional.Conv2D object at 0x7f849875eba8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f849875ecf8> False
<keras.layers.convolutional.Conv2D object at 0x7f849875ee80> False
<keras.layers.convolutional.Conv2D object at 0x7f849875ef60> False
<keras.layers.convolutional.Conv2D object at 0x7f8498770278> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8498770400> False
<keras.layers.convolutional.Conv2D object at 0x7f84987704e0> False
<keras.layers.convolutional.Conv2D object at 0x7f8498770630> False
<keras.layers.convolutional.Conv2D object at 0x7f84987707b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f8498770940> Fa

In [19]:
# Fine-tuning optimizer
# optm = optimizers.RMSprop(lr=1e-4)
optm = optimizers.Nadam(lr=0.00001)

model.compile(loss='categorical_crossentropy',
              optimizer=optm,
              metrics=['acc'])

# Train the model
history = loaded_model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size/10,
      epochs=5*10,
      class_weight=class_weight,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size/2,
      verbose=1,
      callbacks=[keras.callbacks.ModelCheckpoint('learnopencv_vgg5.h5', monitor='val_acc', verbose=2, save_best_only=True, mode='auto', period=1)])
 
# Save the model
model.save('learnopencv_vgg_final5.h5')

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/50
193/192 [==============================] - 1419s 7s/step - loss: 1.2813 - acc: 0.6734 - val_loss: 1.5024 - val_acc: 0.6278

Epoch 00001: val_acc improved from -inf to 0.62785, saving model to learnopencv_vgg5.h5
Epoch 2/50
 26/192 [===>..........................] - ETA: 13:15 - loss: 1.2967 - acc: 0.6696

KeyboardInterrupt: 

### Accumulate predictions and true labels. Using Generator to get the data

In [352]:
evaluated = 0
y_true = []
# y_pred = np.empty(shape=(10,128))
y_pred = []
validation_generator.reset()
generator_names = validation_generator.filenames
for index, (x_val, y_val) in enumerate(validation_generator):
    if(len(y_pred) <= 0):
        y_pred = loaded_model.predict(x_val)
    else:
        y_pred = np.concatenate([y_pred, loaded_model.predict(x_val)])
    y_true = np.concatenate([y_true, np.argmax(y_val, axis=1)])
    evaluated += len(x_val)
    print(evaluated, end='\r')
    
    if(evaluated >= validation_generator.samples/2):
        break
        


IndexError: index 3160 is out of bounds for axis 0 with size 3160

In [385]:
the_map = validation_generator.class_indices
inv_map = {v: int(k) for k, v in validation_generator.class_indices.items()}

y_pred_dict = {file: (inv_map[np.argmax(y_pred[i])], inv_map[int(y_true[i])]) for i, file in enumerate(generator_names[:len(y_pred)])}

In [386]:
inv_map

{0: 1,
 1: 10,
 2: 100,
 3: 101,
 4: 102,
 5: 103,
 6: 104,
 7: 105,
 8: 106,
 9: 107,
 10: 108,
 11: 109,
 12: 11,
 13: 110,
 14: 111,
 15: 112,
 16: 113,
 17: 114,
 18: 115,
 19: 116,
 20: 117,
 21: 118,
 22: 119,
 23: 12,
 24: 120,
 25: 121,
 26: 122,
 27: 123,
 28: 124,
 29: 125,
 30: 126,
 31: 127,
 32: 128,
 33: 13,
 34: 14,
 35: 15,
 36: 16,
 37: 17,
 38: 18,
 39: 19,
 40: 2,
 41: 20,
 42: 21,
 43: 22,
 44: 23,
 45: 24,
 46: 25,
 47: 26,
 48: 27,
 49: 28,
 50: 29,
 51: 3,
 52: 30,
 53: 31,
 54: 32,
 55: 33,
 56: 34,
 57: 35,
 58: 36,
 59: 37,
 60: 38,
 61: 39,
 62: 4,
 63: 40,
 64: 41,
 65: 42,
 66: 43,
 67: 44,
 68: 45,
 69: 46,
 70: 47,
 71: 48,
 72: 49,
 73: 5,
 74: 50,
 75: 51,
 76: 52,
 77: 53,
 78: 54,
 79: 55,
 80: 56,
 81: 57,
 82: 58,
 83: 59,
 84: 6,
 85: 60,
 86: 61,
 87: 62,
 88: 63,
 89: 64,
 90: 65,
 91: 66,
 92: 67,
 93: 68,
 94: 69,
 95: 7,
 96: 70,
 97: 71,
 98: 72,
 99: 73,
 100: 74,
 101: 75,
 102: 76,
 103: 77,
 104: 78,
 105: 79,
 106: 8,
 107: 80,
 108: 81,

In [391]:
all_files = list(y_pred_dict.keys())

In [394]:
pred1 = []
pred2 = []
for i, file in enumerate(all_files):
    pred1.append(y_pred_dict[file])
    pred2.append(y_pred2_dict[file])
    print(i, end='\r')

KeyError: '101/4745.jpeg'

In [351]:
validation_generator.classes

array([  0,   0,   0, ..., 127, 127, 127], dtype=int32)

In [ ]:
y_true

In [ ]:
np.argmax

In [140]:
np.argmax(y_pred, axis=1)

array([ 10, 109,  17, ...,   8,  17, 126])

array([8., 8., 9., 9., 9., 9., 9., 9., 9., 9.])

In [301]:
import glob
import pickle
import numpy as np

FEATURES_LOCATION = '/furniture-data/features/'
F_CORE = 'cnn_features_'

def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode, load_n):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    filenames = []
    for key, value in list(data.items()):
        the_class = key
        features = np.array(list(value.values()))
        names = list(value.keys())
        for i, feature in enumerate(features):
            y.append(the_class)
            X.append(feature)
            filenames.append(names[i])

    return X[:load_n], y[:load_n], filenames[:load_n]

In [302]:
X_val, Y_val, Y_val_names = load_data('valid', len(y_true))
X_val

[array([0.23763506, 0.02864071, 0.2449646 , ..., 0.5705015 , 2.7354367 ,
        0.16989067], dtype=float32),
 array([0.        , 0.05428545, 0.03587624, ..., 0.6735942 , 0.11410042,
        0.10808976], dtype=float32),
 array([0.6102923 , 0.0844544 , 0.8667229 , ..., 0.02681307, 0.31706965,
        0.        ], dtype=float32),
 array([0.8863657 , 0.4207272 , 0.        , ..., 0.04117322, 0.37698755,
        0.        ], dtype=float32),
 array([0.21726106, 0.1036961 , 1.0583332 , ..., 0.26514274, 0.00766741,
        0.14100602], dtype=float32),
 array([0.13975202, 0.        , 0.01278213, ..., 0.        , 0.        ,
        0.0394525 ], dtype=float32),
 array([0.0620111 , 0.1721003 , 0.19316922, ..., 0.00226412, 0.        ,
        0.51825756], dtype=float32),
 array([0.13793196, 0.00313238, 0.29283962, ..., 0.        , 0.        ,
        0.9668553 ], dtype=float32),
 array([0.02940386, 0.        , 0.03610207, ..., 0.0348406 , 0.10597557,
        0.37248743], dtype=float32),
 array([0.

In [347]:
y_pred2 = model.predict(np.array(X_val))

In [389]:
y_pred2_dict = {file: (np.argmax(y_pred2[i]), Y_val[i]) for i, file in enumerate(Y_val_names)}

In [303]:
def combine(a_list):
    return a_list[0] + '/' + a_list[1]

Y_val_names = [combine(name.replace('\\', '/').split('/')[-2:]) for name in Y_val_names]
Y_val_names

['1/3181.jpeg',
 '1/2620.jpeg',
 '1/3858.jpeg',
 '1/3758.jpeg',
 '1/1471.jpeg',
 '1/3067.jpeg',
 '1/2593.jpeg',
 '1/3300.jpeg',
 '1/1831.jpeg',
 '1/29.jpeg',
 '1/4693.jpeg',
 '1/4740.jpeg',
 '1/4067.jpeg',
 '1/50.jpeg',
 '1/3990.jpeg',
 '1/4122.jpeg',
 '1/2065.jpeg',
 '1/5425.jpeg',
 '1/426.jpeg',
 '1/6315.jpeg',
 '1/1589.jpeg',
 '1/3699.jpeg',
 '1/4869.jpeg',
 '1/4635.jpeg',
 '1/103.jpeg',
 '1/3648.jpeg',
 '1/4465.jpeg',
 '1/5125.jpeg',
 '1/2357.jpeg',
 '1/5665.jpeg',
 '1/3038.jpeg',
 '1/4293.jpeg',
 '1/289.jpeg',
 '1/4325.jpeg',
 '1/4945.jpeg',
 '1/2592.jpeg',
 '1/977.jpeg',
 '1/30.jpeg',
 '1/2523.jpeg',
 '1/4851.jpeg',
 '1/2512.jpeg',
 '1/2150.jpeg',
 '1/3510.jpeg',
 '1/5624.jpeg',
 '1/2491.jpeg',
 '1/5530.jpeg',
 '1/2308.jpeg',
 '1/5631.jpeg',
 '1/5768.jpeg',
 '1/1525.jpeg',
 '2/3493.jpeg',
 '2/4613.jpeg',
 '2/2048.jpeg',
 '2/33.jpeg',
 '2/1316.jpeg',
 '2/4396.jpeg',
 '2/4402.jpeg',
 '2/5140.jpeg',
 '2/5112.jpeg',
 '2/3223.jpeg',
 '2/4879.jpeg',
 '2/199.jpeg',
 '2/1339.jpeg',
 '2/3

In [345]:
Y_val_names_sorted[199] == generator_names[199]

False

In [305]:
indexes = list(range(len(Y_val_names)))

order = sorted(zip(Y_val_names, Y_val, X_val), key=lambda name: generator_names.index(name[0]))

Y_val_names_sorted = [el[0] for el in order]
Y_val_sorted = [el[1] for el in order]
X_val_sorted = [el[2] for el in order]

In [224]:
loaded_model2 = keras.models.load_model('./ValAcc79410.h5')

In [225]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(2048,)))
model.add(loaded_model2.layers[-3])
model.add(loaded_model2.layers[-2])
model.add(loaded_model2.layers[-1])

In [226]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_18 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               131200    
Total params: 2,229,376
Trainable params: 2,229,376
Non-trainable params: 0
_________________________________________________________________


In [254]:
np.array(X_val_sorted).reshape(-1,1).shape

(6309, 1)

In [306]:
y_pred2 = model.predict(np.array(X_val_sorted))

In [228]:
y_pred2.shape

(6309, 128)

In [332]:
[the_map[str(np.argmax(y) + 1)] + 1 for y in y_pred]

[1,
 1,
 1,
 1,
 1,
 62,
 89,
 1,
 1,
 1,
 89,
 1,
 1,
 1,
 47,
 1,
 113,
 1,
 58,
 1,
 54,
 1,
 113,
 1,
 1,
 61,
 28,
 1,
 1,
 112,
 45,
 1,
 1,
 1,
 1,
 68,
 1,
 30,
 1,
 1,
 1,
 1,
 113,
 1,
 113,
 113,
 1,
 1,
 113,
 1,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 87,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 61,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 63,
 41,
 41,
 41,
 18,
 41,
 41,
 41,
 18,
 66,
 100,
 52,
 52,
 47,
 32,
 100,
 52,
 66,
 97,
 52,
 52,
 73,
 119,
 92,
 94,
 52,
 100,
 52,
 100,
 52,
 66,
 52,
 100,
 52,
 52,
 47,
 52,
 52,
 52,
 66,
 98,
 52,
 53,
 52,
 52,
 66,
 52,
 100,
 100,
 52,
 100,
 52,
 100,
 52,
 47,
 52,
 66,
 100,
 85,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 49,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 70,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 33,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 63,
 74,
 74,
 74,
 68,
 74,
 90,
 80,
 74,
 74,

In [335]:
[np.argmax(y) + 1 for y in y_pred2]

[18,
 1,
 1,
 18,
 1,
 1,
 6,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 55,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 17,
 6,
 1,
 1,
 59,
 6,
 1,
 1,
 1,
 1,
 17,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 6,
 1,
 1,
 1,
 1,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 99,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 35,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 69,
 11,
 11,
 11,
 40,
 50,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 78,
 78,
 11,
 11,
 11,
 11,
 11,
 56,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 20,
 11,
 77,
 12,
 12,
 12,
 37,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 112,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 122,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 31,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 1

In [72]:
np.argmax(y_pred2, axis=1)

array([  0,   0,   5, ...,  79, 127, 127])

In [103]:
y_val

array([  1,   1,   1, ..., 128, 128, 128])

In [307]:
def get_acc(y_pred, y_true):
    print(len(y_pred))
    print(len(y_true))
    return np.sum(y_pred == y_true) / len(y_pred)

get_acc(np.argmax(y_pred2, axis=1) + 1, Y_val_sorted)

3160
3160


0.7715189873417722

In [333]:
get_acc([the_map[str(np.argmax(y) + 1)] for y in y_pred], Y_val_sorted)

3160
3160


0.0

In [329]:
validation_generator.class_indices
the_map = validation_generator.class_indices
inv_map = {v: k for k, v in validation_generator.class_indices.items()}
the_map

{'1': 0,
 '10': 1,
 '100': 2,
 '101': 3,
 '102': 4,
 '103': 5,
 '104': 6,
 '105': 7,
 '106': 8,
 '107': 9,
 '108': 10,
 '109': 11,
 '11': 12,
 '110': 13,
 '111': 14,
 '112': 15,
 '113': 16,
 '114': 17,
 '115': 18,
 '116': 19,
 '117': 20,
 '118': 21,
 '119': 22,
 '12': 23,
 '120': 24,
 '121': 25,
 '122': 26,
 '123': 27,
 '124': 28,
 '125': 29,
 '126': 30,
 '127': 31,
 '128': 32,
 '13': 33,
 '14': 34,
 '15': 35,
 '16': 36,
 '17': 37,
 '18': 38,
 '19': 39,
 '2': 40,
 '20': 41,
 '21': 42,
 '22': 43,
 '23': 44,
 '24': 45,
 '25': 46,
 '26': 47,
 '27': 48,
 '28': 49,
 '29': 50,
 '3': 51,
 '30': 52,
 '31': 53,
 '32': 54,
 '33': 55,
 '34': 56,
 '35': 57,
 '36': 58,
 '37': 59,
 '38': 60,
 '39': 61,
 '4': 62,
 '40': 63,
 '41': 64,
 '42': 65,
 '43': 66,
 '44': 67,
 '45': 68,
 '46': 69,
 '47': 70,
 '48': 71,
 '49': 72,
 '5': 73,
 '50': 74,
 '51': 75,
 '52': 76,
 '53': 77,
 '54': 78,
 '55': 79,
 '56': 80,
 '57': 81,
 '58': 82,
 '59': 83,
 '6': 84,
 '60': 85,
 '61': 86,
 '62': 87,
 '63': 88,
 '64': 8

In [338]:
np.array([int(inv_map[y]) for y in (y_true)])

array([ 1,  1,  1, ..., 41, 41, 41])

In [340]:
np.array(Y_val_sorted)

array([1, 1, 1, ..., 9, 9, 9])

In [298]:
Y_val_names_sorted

['1/103.jpeg',
 '1/1471.jpeg',
 '1/1525.jpeg',
 '1/1589.jpeg',
 '1/1831.jpeg',
 '1/2065.jpeg',
 '1/2150.jpeg',
 '1/2308.jpeg',
 '1/2357.jpeg',
 '1/2491.jpeg',
 '1/2512.jpeg',
 '1/2523.jpeg',
 '1/2592.jpeg',
 '1/2593.jpeg',
 '1/2620.jpeg',
 '1/289.jpeg',
 '1/29.jpeg',
 '1/30.jpeg',
 '1/3038.jpeg',
 '1/3067.jpeg',
 '1/3181.jpeg',
 '1/3300.jpeg',
 '1/3510.jpeg',
 '1/3648.jpeg',
 '1/3699.jpeg',
 '1/3758.jpeg',
 '1/3858.jpeg',
 '1/3990.jpeg',
 '1/4067.jpeg',
 '1/4122.jpeg',
 '1/426.jpeg',
 '1/4293.jpeg',
 '1/4325.jpeg',
 '1/4465.jpeg',
 '1/4635.jpeg',
 '1/4693.jpeg',
 '1/4740.jpeg',
 '1/4851.jpeg',
 '1/4869.jpeg',
 '1/4945.jpeg',
 '1/50.jpeg',
 '1/5125.jpeg',
 '1/5425.jpeg',
 '1/5530.jpeg',
 '1/5624.jpeg',
 '1/5631.jpeg',
 '1/5665.jpeg',
 '1/5768.jpeg',
 '1/6315.jpeg',
 '1/977.jpeg',
 '10/1000.jpeg',
 '10/1019.jpeg',
 '10/108.jpeg',
 '10/1092.jpeg',
 '10/1131.jpeg',
 '10/1194.jpeg',
 '10/1238.jpeg',
 '10/1251.jpeg',
 '10/1366.jpeg',
 '10/1412.jpeg',
 '10/1511.jpeg',
 '10/1587.jpeg',
 '10/17

In [297]:
validation_generator.filenames

['1/103.jpeg',
 '1/1471.jpeg',
 '1/1525.jpeg',
 '1/1589.jpeg',
 '1/1831.jpeg',
 '1/2065.jpeg',
 '1/2150.jpeg',
 '1/2308.jpeg',
 '1/2357.jpeg',
 '1/2491.jpeg',
 '1/2512.jpeg',
 '1/2523.jpeg',
 '1/2592.jpeg',
 '1/2593.jpeg',
 '1/2620.jpeg',
 '1/289.jpeg',
 '1/29.jpeg',
 '1/30.jpeg',
 '1/3038.jpeg',
 '1/3067.jpeg',
 '1/3181.jpeg',
 '1/3300.jpeg',
 '1/3510.jpeg',
 '1/3648.jpeg',
 '1/3699.jpeg',
 '1/3758.jpeg',
 '1/3858.jpeg',
 '1/3990.jpeg',
 '1/4067.jpeg',
 '1/4122.jpeg',
 '1/426.jpeg',
 '1/4293.jpeg',
 '1/4325.jpeg',
 '1/4465.jpeg',
 '1/4635.jpeg',
 '1/4693.jpeg',
 '1/4740.jpeg',
 '1/4851.jpeg',
 '1/4869.jpeg',
 '1/4945.jpeg',
 '1/50.jpeg',
 '1/5125.jpeg',
 '1/5425.jpeg',
 '1/5530.jpeg',
 '1/5624.jpeg',
 '1/5631.jpeg',
 '1/5665.jpeg',
 '1/5768.jpeg',
 '1/6315.jpeg',
 '1/977.jpeg',
 '10/1000.jpeg',
 '10/1019.jpeg',
 '10/108.jpeg',
 '10/1092.jpeg',
 '10/1131.jpeg',
 '10/1194.jpeg',
 '10/1238.jpeg',
 '10/1251.jpeg',
 '10/1366.jpeg',
 '10/1412.jpeg',
 '10/1511.jpeg',
 '10/1587.jpeg',
 '10/17